# MDX-Net Colab
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://github.githubassets.com/pinned-octocat.svg" alt="icon1" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>Trained models provided in this notebook are from <a href="https://github.com/Anjok07">UVR-GUI</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://github.com/Anjok07/ultimatevocalremovergui/raw/master/gui_data/img/GUI-Icon.ico" alt="icon2" style="margin-right:10px; height: 20px;margin-top:10px" width="1.5%">
  <span>OFFICIAL UVR GITHUB PAGE: <a href="https://github.com/Anjok07/ultimatevocalremovergui">here</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://avatars.githubusercontent.com/u/24620594" alt="icon3" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>OFFICIAL CLI Version: <a href="https://github.com/tsurumeso/vocal-remover">here</a>.</span>
</div>
<div style="display:flex; align-items:center; font-size: 16px;">
  <img src="https://icons.getbootstrap.com/assets/icons/discord.svg" alt="icon4" style="margin-right:10px; height: 20px;" width="1.5%">
  <span>Join our <a href="https://cutt.ly/0TcDjmo">Discord server</a>!</span>
</div>
<sup><br>Ultimate Vocal Remover (unofficial)</sup>
<sup><br>MDX-Net by <a href="https://github.com/kuielab">kuielab</a> and adapted for Colaboratory by <a href="https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw">AudioHacker</a>.</sup>


In [ ]:
import json
import os
import os.path
import psutil
import requests
import subprocess
import time
import sys
from bs4 import BeautifulSoup
from google.colab import drive, files, output
from IPython.display import Audio, display

if "first_cell_ran" in locals():
    print("You've ran this cell for this session. No need to run it again.\nif you think something went wrong or you want to change mounting path, restart the runtime.")
else:

    print('Setting up... please wait around 1-2 minute(s).')

    branch = "https://github.com/NaJeongMo/Colab-for-MDX_B"

    model_params = "https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/model_data.json"
    _Models = "https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/"
    _models = "https://pastebin.com/raw/jBzYB8vz"

    arl_check_endpoint = 'https://dz.doubledouble.top/check' # param: arl?=<>

    file_folder = "Colab-for-MDX_B"

    model_ids = requests.get(_models).json()
    model_params = requests.get(model_params).json()

    os.makedirs("tmp_models", exist_ok=True)

    # @markdown If you don't wish to mount google drive, uncheck this box.
    MountDrive = False  # @param{type:"boolean"}
    # @markdown The path for the drive to be mounted: Please be cautious when modifying this as it can cause issues if not done properly.
    mounting_path = "/content/drive/MyDrive"  # @param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
    # @markdown Force update and disregard local changes: discards all local modifications in your repository, effectively replacing all files with the versions from the original commit.
    force_update = False  # @param{type:"boolean"}
    # @markdown Auto Update (does not discard your changes)
    auto_update = True  # @param{type:"boolean"}


    reqs_apt = []  # !sudo apt-get install
    reqs_pip = ["librosa>=0.6.3,<0.9", "onnxruntime_gpu", "deemix", "yt_dlp"]  # pip3 install

    class hide_opt:  # hide outputs
        def __enter__(self):
            self._original_stdout = sys.stdout
            sys.stdout = open(os.devnull, "w")

        def __exit__(self, exc_type, exc_val, exc_tb):
            sys.stdout.close()
            sys.stdout = self._original_stdout

    def get_size(bytes, suffix="B"):  # read ram
        global svmem
        factor = 1024
        for unit in ["", "K", "M", "G", "T", "P"]:
            if bytes < factor:
                return f"{bytes:.2f}{unit}{suffix}"
            bytes /= factor
        svmem = psutil.virtual_memory()


    print('installing requirements...',end=' ')
    with hide_opt():
        for x in reqs_apt:
            subprocess.run(["sudo", "apt-get", "install", x])
        for x in reqs_pip:
            subprocess.run(["python3", "-m", "pip", "install", x])
    print('done')

    def install_or_mount_drive():
        print(
            "Please log in to your account by following the prompts in the pop-up tab.\nThis step is necessary to install the files to your Google Drive.\nIf you have any concerns about the safety of this notebook, you can choose not to mount your drive by unchecking the \"MountDrive\" checkbox."
        )
        drive.mount("/content/drive", force_remount=True)
        os.chdir(mounting_path)
        # check if previous installation is done
        if os.path.exists(os.path.join(mounting_path, file_folder)):
            # update checking
            os.chdir(file_folder)

            if force_update:
                print('Force updating...')

                commands = [
                    ["git", "pull"],
                    ["git", "checkout", "--", "."],
                ]

                for cmd in commands:
                    subprocess.run(cmd)

            elif auto_update:
                print('Checking for updates...')
                commands = [
                    ["git", "pull"],
                ]

                for cmd in commands:
                    subprocess.run(cmd)
        else:
            subprocess.run(["git", "clone", "https://github.com/NaJeongMo/Colab-for-MDX_B.git"])
            os.chdir(file_folder)

    def use_uvr_without_saving():
        global mounting_path
        print("Notice: files won't be saved to personal drive.")
        print(f"Downloading {file_folder}...", end=" ")
        mounting_path = "/content"
        with hide_opt():
            os.chdir(mounting_path)
            subprocess.run(["git", "clone", "https://github.com/NaJeongMo/Colab-for-MDX_B.git"])
            os.chdir(file_folder)

    if MountDrive:
        install_or_mount_drive()
    else:
        use_uvr_without_saving()
    print("done!")
    if not os.path.exists("tracks"):
        os.mkdir("tracks")

    print('Importing required libraries...',end=' ')

    import os
    import mdx
    import librosa
    import torch
    import soundfile as sf
    import yt_dlp

    from deezer import Deezer
    from deezer import TrackFormats
    import deemix
    from deemix.settings import load as loadSettings
    from deemix.downloader import Downloader
    from deemix import generateDownloadObject

    def id_to_ptm(mkey):
        if mkey in model_ids:
            mpath = f"/content/tmp_models/{mkey}"
            if not os.path.exists(f'/content/tmp_models/{mkey}'):
                print('Downloading model...',end=' ')
                subprocess.run(
                    ["wget", _Models+mkey, "-O", mpath]
                )
                print(f'saved to {mpath}')
                # get_ipython().system(f'gdown {model_id} -O /content/tmp_models/{mkey}')
                return mpath
            else:
                return mpath
        else:
            mpath = f'models/{mkey}'
            return mpath

    def prepare_mdx(custom_param=False, dim_f=None, dim_t=None, n_fft=None, stem_name=None):
        device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
        if custom_param:
            assert dim_f is None or dim_t is None or n_fft is None, 'Custom parameter selected, but no parameters are provided.'
            mdx_model = mdx.MDX_Model(
                device,
                dim_f = dim_f,
                dim_t = dim_t,
                n_fft = n_fft,
                stem_name=stem_name
            )
        else:
            model_hash = mdx.MDX.get_hash(onnx)
            if model_hash in model_params:
                mp = model_params.get(model_hash)
                mdx_model = mdx.MDX_Model(
                    device,
                    dim_f = mp["mdx_dim_f_set"],
                    dim_t = 2**mp["mdx_dim_t_set"],
                    n_fft = mp["mdx_n_fft_scale_set"],
                    stem_name=mp["primary_stem"]
                )
        return mdx_model

    def run_mdx(onnx, mdx_model, filename,suffix=None):
        mdx_sess = mdx.MDX(onnx,mdx_model)
        print(f"Processing: {filename}")
        wave, sr = librosa.load(filename,mono=False, sr=44100)
        wave_processed = mdx_sess.process_wave(wave)
        save_path = f"{os.path.basename(os.path.splitext(filename)[0])}_{mdx_model.stem_name if suffix is None else suffix}.wav"
        sf.write(
            os.path.join(
                'separated',
                save_path
            ),
            wave_processed.T,
            sr
        )
        print(f'done, saved to: {save_path}')

    def is_valid_url(url):
        import re
        regex = re.compile(
            r'^https?://'  # http:// or https://
            r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'  # domain...
            r'localhost|'  # localhost...
            r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
            r'(?::\d+)?'  # optional port
            r'(?:/?|[/?]\S+)$', re.IGNORECASE)
        return url is not None and regex.search(url)

    def download_deezer(link, arl, fmt='FLAC'):
        match fmt:
            case 'FLAC':
                bitrate = TrackFormats.FLAC
            case 'MP3_320':
                bitrate = TrackFormats.MP3_320
            case 'MP3_128':
                bitrate = TrackFormats.MP3_128
            case _:
                bitrate = TrackFormats.MP3_128

        dz = Deezer()
        settings = loadSettings('dz_config')
        settings['downloadLocation'] = './tracks'
        if not dz.login_via_arl(arl.strip()):
            raise Exception('Error while logging in with provided ARL.')
        downloadObject = generateDownloadObject(dz, link, bitrate)
        print(f'Downloading {downloadObject.type}: "{downloadObject.title}" by {downloadObject.artist}...',end=' ',flush=True)
        Downloader(dz, downloadObject, settings).start()
        print(f'done.')

        path_to_audio = []
        for file in downloadObject.files:
            path_to_audio.append(file["path"])

        return path_to_audio

    def download_link(url):
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': '%(title)s.%(ext)s',
            'nocheckcertificate': True,
            'ignoreerrors': True,
            'no_warnings': True,
            'extractaudio': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(url, download=True)
            download_path = ydl.prepare_filename(result)
        return download_path


    print('finished setting up!')
    first_cell_ran = True

In [ ]:
if 'first_cell_ran' in locals():
    os.chdir(mounting_path + '/' + file_folder + '/')
    #parameter markdowns-----------------
    #@markdown ### Input files
    #@markdown track filename: Upload your songs to the "tracks" folder. You may provide multiple links/files by spliting them with ;
    filename = "" #@param {type:"string"}
    #@markdown onnx model (if you have your own model, upload it in models folder)
    onnx = "UVR-MDX-NET-Inst_HQ_3.onnx" #@param ["Kim_Inst.onnx", "Kim_Vocal_1.onnx", "Kim_Vocal_2.onnx", "kuielab_a_bass.onnx", "kuielab_a_drums.onnx", "kuielab_a_other.onnx", "kuielab_a_vocals.onnx", "kuielab_b_bass.onnx", "kuielab_b_drums.onnx", "kuielab_b_other.onnx", "kuielab_b_vocals.onnx", "Reverb_HQ_By_FoxJoy.onnx", "UVR-MDX-NET-Inst_1.onnx", "UVR-MDX-NET-Inst_2.onnx", "UVR-MDX-NET-Inst_3.onnx", "UVR-MDX-NET-Inst_HQ_1.onnx", "UVR-MDX-NET-Inst_HQ_2.onnx", "UVR-MDX-NET-Inst_Main.onnx", "UVR_MDXNET_1_9703.onnx", "UVR_MDXNET_2_9682.onnx", "UVR_MDXNET_3_9662.onnx", "UVR_MDXNET_9482.onnx", "UVR_MDXNET_KARA.onnx", "UVR_MDXNET_KARA_2.onnx", "UVR_MDXNET_Main.onnx", "UVR-MDX-NET-Inst_HQ_3.onnx"]{allow-input: true}

    #@markdown ### Custom model parameters
    #@markdown Use custom model parameters. (Default: auto)
    use_custom_parameter = False  # @param{type:"boolean"}
    #@markdown Output file suffix (usually the stem name e.g Vocals)
    suffix = "inst_custom" #@param {type:"string"}
    #@markdown Model parameters
    dim_f = 2048 #@param {type: "integer"}
    dim_t = 8 #@param {type: "integer"}
    n_fft = 5120 #@param {type: "integer"}

    #@markdown ### Extras
    #@markdown Deezer arl: paste your ARL here for deezer tracks directly!
    arl = "" #@param {type:"string"}
    #@markdown Track format: select track quality/format
    track_format = "FLAC" #@param ["FLAC","MP3_320","MP3_128"]



    onnx = id_to_ptm(onnx)

    mdx_model = prepare_mdx(use_custom_parameter, dim_f, dim_t, n_fft)

    filename_split = filename.split(';')

    usable_files = []
    for fn in filename_split:
        fn = fn.strip()
        if is_valid_url(fn):
            dm, ltype, lid = deemix.parseLink(fn)
            if ltype and lid:
                usable_files += download_deezer(fn, arl, track_format)
            else:
                usable_files+=[download_link(fn)]
        else:
            usable_files.append(os.path.join('tracks',fn))
    for filename in usable_files:
        run_mdx(onnx, mdx_model, filename,suffix=suffix if use_custom_parameter else None)



    # if use_custom_parameter:
    #     mdx_model = mdx.MDX_Model(
    #         torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu'),
    #         dim_f = dim_f,
    #         dim_t = dim_t,
    #         n_fft = n_fft,
    #     )
    # else:
    #     model_hash = mdx.MDX.get_hash(onnx)
    #     if model_hash in model_params:
    #         mp = model_params.get(model_hash)
    #         mdx_model = mdx.MDX_Model(
    #             torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu'),
    #             dim_f = mp["mdx_dim_f_set"],
    #             dim_t = 2**mp["mdx_dim_t_set"],
    #             n_fft = mp["mdx_n_fft_scale_set"]
    #         )
    #     else:
    #         raise Exception(f'Not proceeding; hash "{model_hash}" not found in model_data.json\nPlease use custom parameter.')

    # mdx_sess = mdx.MDX(onnx,mdx_model)
    # print(f"Processing: {filename}")
    # wave, sr = librosa.load(os.path.join('tracks',filename),mono=False, sr=44100)
    # wave_processed = mdx_sess.process_wave(wave)
    # print('Done!')
    # sf.write(
    #     f'processed_{filename}.wav',
    #     wave_processed.T,
    #     sr
    # )
